# Chapter 13: Web3.js and ethers.js Libraries

---

## 13.1 Introduction to Web3 Libraries

### 13.1.1 Why Web3 Libraries are Needed

Smart contracts alone are just code sitting on the blockchain. To build a user-facing application, you need a way for your frontend (or backend) to interact with these contracts—to read data, send transactions, and listen to events. This is where Web3 libraries come in.

**Web3 libraries** provide a JavaScript (and TypeScript) interface to the Ethereum blockchain. They abstract away the complexities of RPC calls, transaction signing, and data encoding, allowing developers to focus on application logic.

**What Web3 libraries do:**
- Connect to Ethereum nodes (via RPC, WebSocket, or IPC).
- Manage accounts and sign transactions.
- Encode/decode function calls and return values (using ABIs).
- Estimate gas and send transactions.
- Listen to contract events.
- Provide utility functions (converting units, hashing, etc.).

Without these libraries, you'd have to manually construct JSON-RPC payloads, handle hexadecimal encoding, and manage nonces—a tedious and error-prone process.

### 13.1.2 Web3.js vs. ethers.js Comparison

The two most popular JavaScript libraries for Ethereum are **web3.js** and **ethers.js**. Both are powerful, but they have different design philosophies and feature sets.

```
┌────────────────────────────────────┬────────────────────────────────────┐
│            web3.js                  │              ethers.js             │
├────────────────────────────────────┼────────────────────────────────────┤
│ Maintained by the Ethereum         │ Maintained by Richard Moore        │
│ Foundation and community            │ (ethers.io) and community          │
├────────────────────────────────────┼────────────────────────────────────┤
│ Larger, more comprehensive          │ Smaller, modular, tree-shakeable   │
│ bundle size                         │                                   │
├────────────────────────────────────┼────────────────────────────────────┤
│ Multiple ways to do the same thing  │ More opinionated, one clear way    │
│ (can be confusing)                  │                                   │
├────────────────────────────────────┼────────────────────────────────────┤
│ Uses callback patterns and          │ Uses Promises and async/await      │
│ Promises (mixed)                    │ consistently                       │
├────────────────────────────────────┼────────────────────────────────────┤
│ Provider can be set/changed         │ Provider is immutable once set     │
│ dynamically                         │                                   │
├────────────────────────────────────┼────────────────────────────────────┤
│ Native support for some Ethereum    │ Better ENS support (built-in)      │
│ Name Service (ENS) limited          │                                   │
├────────────────────────────────────┼────────────────────────────────────┤
│ Larger community, more tutorials    │ Growing community, excellent       │
│                                     │ documentation                      │
└────────────────────────────────────┴────────────────────────────────────┘
```

**When to use which:**
- **ethers.js** is generally recommended for new projects due to its cleaner API, better documentation, and smaller size.
- **web3.js** is still widely used, especially in older codebases and when you need features not yet in ethers.js.

In this chapter, we'll focus primarily on **ethers.js**, as it has become the de facto standard for modern DApp development. We'll also cover the basics of web3.js for completeness.

---

## 13.2 ethers.js Deep Dive

ethers.js aims to be a complete and compact library for interacting with the Ethereum blockchain. It's designed with security, simplicity, and size in mind.

### 13.2.1 Installation and Setup

**Install via npm:**
```bash
npm install ethers
```

**In a browser (using a CDN):**
```html
<script src="https://cdn.ethers.io/lib/ethers-5.7.umd.min.js"></script>
```

**Import in your project:**
```javascript
// CommonJS
const { ethers } = require('ethers');

// ES6 modules
import { ethers } from 'ethers';
```

**Basic structure:**
ethers.js is organized into several modules:
- `ethers.providers`: Connect to Ethereum nodes.
- `ethers.Wallet`: Manage accounts and signing.
- `ethers.Contract`: Interact with smart contracts.
- `ethers.utils`: Utility functions (formatting, hashing, etc.).

### 13.2.2 Providers: Connecting to Ethereum

A **Provider** is an abstraction of a connection to the Ethereum network. It provides read-only access to blockchain data.

**Built-in provider types:**
- `JsonRpcProvider`: Connects to a JSON-RPC endpoint (like Infura, Alchemy, or a local node).
- `Web3Provider`: Wraps an injected web3 provider (like MetaMask).
- `FallbackProvider`: Automatically falls back to multiple providers.
- `AlchemyProvider`, `InfuraProvider`: Pre-configured for popular services.

**Example: Connecting to a public RPC**
```javascript
const { ethers } = require('ethers');

// Connect to Ethereum mainnet via a public RPC
const provider = new ethers.providers.JsonRpcProvider('https://eth-mainnet.g.alchemy.com/v2/YOUR_API_KEY');

// Or use Infura
const provider = new ethers.providers.InfuraProvider('mainnet', 'YOUR_INFURA_PROJECT_ID');

// Get block number
async function getBlockNumber() {
  const blockNumber = await provider.getBlockNumber();
  console.log('Current block number:', blockNumber);
}
```

**Connecting to a local network (Hardhat/Ganache):**
```javascript
const provider = new ethers.providers.JsonRpcProvider('http://127.0.0.1:8545');
```

**Using MetaMask's provider:**
```javascript
// In a browser
const provider = new ethers.providers.Web3Provider(window.ethereum);
await provider.send('eth_requestAccounts', []); // Request user accounts
```

### 13.2.3 Signers: Managing Accounts

A **Signer** is an object that can sign transactions and messages. It's associated with a specific private key and can send transactions that modify state.

**Types of signers:**
- `Wallet`: A signer backed by a private key.
- `JsonRpcSigner`: A signer connected to a provider (e.g., from MetaMask).

**Creating a wallet from a private key:**
```javascript
const privateKey = '0x0123456789abcdef...';
const wallet = new ethers.Wallet(privateKey, provider);
```

**Creating a wallet from a mnemonic:**
```javascript
const mnemonic = 'olympic ...';
const wallet = ethers.Wallet.fromMnemonic(mnemonic);
```

**Getting a signer from a Web3Provider (MetaMask):**
```javascript
const provider = new ethers.providers.Web3Provider(window.ethereum);
const signer = provider.getSigner(); // Default account
```

**Signer methods:**
- `signer.getAddress()`: Get the account address.
- `signer.signMessage(message)`: Sign a message.
- `signer.sendTransaction(tx)`: Send a transaction.
- `signer.getBalance()`: Get account balance.

### 13.2.4 Contracts: Interacting with Smart Contracts

The `Contract` class in ethers.js is used to interact with deployed smart contracts. You need the contract's **address** and **ABI** (Application Binary Interface).

**Creating a contract instance:**
```javascript
const contractAddress = '0x...';
const contractABI = [ /* ABI array */ ];

// For read-only operations (no signer needed)
const contract = new ethers.Contract(contractAddress, contractABI, provider);

// For transactions (needs a signer)
const contractWithSigner = new ethers.Contract(contractAddress, contractABI, signer);
```

**Calling read-only functions (view/pure):**
```javascript
const name = await contract.name();
const totalSupply = await contract.totalSupply();
const balance = await contract.balanceOf('0x...');
```

**Sending transactions (state-changing functions):**
```javascript
const tx = await contractWithSigner.transfer('0x...', ethers.utils.parseEther('1.0'));
await tx.wait(); // Wait for transaction to be mined
console.log('Transaction hash:', tx.hash);
```

**Sending ETH with a function (payable):**
```javascript
const tx = await contractWithSigner.buyTokens({
  value: ethers.utils.parseEther('0.1')
});
```

### 13.2.5 Utilities: Formatting and Conversions

ethers.utils provides many helper functions.

**Ether and Wei conversion:**
```javascript
// 1 ether = 10^18 wei
const amountInWei = ethers.utils.parseEther('1.5'); // '1500000000000000000'
const amountInEther = ethers.utils.formatEther('1500000000000000000'); // '1.5'
```

**Other units:**
```javascript
ethers.utils.parseUnits('123.45', 'ether'); // same as parseEther
ethers.utils.parseUnits('123.45', 'gwei');
ethers.utils.formatUnits('123000000000', 'gwei');
```

**Hashing:**
```javascript
const hash = ethers.utils.id('Hello World'); // keccak256 of UTF-8 bytes
const solidityHash = ethers.utils.solidityKeccak256(['string'], ['Hello World']);
```

**Signing and verifying messages:**
```javascript
const message = 'Hello';
const signature = await signer.signMessage(message);

const recoveredAddress = ethers.utils.verifyMessage(message, signature);
console.log(recoveredAddress === signer.address); // true
```

**ABI encoding/decoding:**
```javascript
const iface = new ethers.utils.Interface(abi);
const data = iface.encodeFunctionData('transfer', [to, amount]);
const decoded = iface.decodeFunctionResult('transfer', result);
```

---

## 13.3 Web3.js Deep Dive

web3.js is the original JavaScript library for Ethereum. While newer projects often choose ethers.js, understanding web3.js is valuable for maintaining legacy code and because some tools still rely on it.

### 13.3.1 Installation and Setup

**Install via npm:**
```bash
npm install web3
```

**In a browser (using a CDN):**
```html
<script src="https://cdn.jsdelivr.net/npm/web3@4.0.1/dist/web3.min.js"></script>
```

**Import:**
```javascript
const Web3 = require('web3');
// or
import Web3 from 'web3';
```

### 13.3.2 Web3 Provider Configuration

Web3 needs a provider to communicate with the blockchain.

**HTTP Provider:**
```javascript
const web3 = new Web3('https://eth-mainnet.g.alchemy.com/v2/YOUR_API_KEY');
```

**WebSocket Provider (for subscriptions):**
```javascript
const web3 = new Web3('wss://eth-mainnet.g.alchemy.com/v2/YOUR_API_KEY');
```

**Using MetaMask's provider:**
```javascript
if (window.ethereum) {
  const web3 = new Web3(window.ethereum);
  await window.ethereum.request({ method: 'eth_requestAccounts' });
} else {
  console.log('Please install MetaMask');
}
```

### 13.3.3 Contract Abstraction

Similar to ethers.js, web3.js provides a contract abstraction.

**Creating a contract instance:**
```javascript
const contractABI = [ /* ABI */ ];
const contractAddress = '0x...';
const contract = new web3.eth.Contract(contractABI, contractAddress);
```

**Calling functions:**
- `contract.methods.functionName().call()` for read-only.
- `contract.methods.functionName().send()` for transactions.

**Example:**
```javascript
// Read
const name = await contract.methods.name().call();

// Write (needs account)
const accounts = await web3.eth.getAccounts();
await contract.methods.transfer('0x...', '1000').send({ from: accounts[0] });
```

**Sending ETH with a payable function:**
```javascript
await contract.methods.buyTokens().send({
  from: accounts[0],
  value: web3.utils.toWei('0.1', 'ether')
});
```

### 13.3.4 Event Subscriptions

Web3.js supports real-time event listening via WebSocket.

**Subscribe to contract events:**
```javascript
contract.events.Transfer({
  filter: { from: accounts[0] }, // optional
  fromBlock: 'latest'
})
.on('data', event => {
  console.log('Transfer event:', event.returnValues);
})
.on('error', console.error);
```

**Subscribe to new blocks:**
```javascript
web3.eth.subscribe('newBlockHeaders', (error, blockHeader) => {
  if (!error) {
    console.log('New block:', blockHeader.number);
  }
});
```

**Unsubscribe:**
```javascript
subscription.unsubscribe();
```

---

## 13.4 Code Examples

Let's build a series of practical examples using ethers.js (since it's our primary focus). These examples assume you have a browser environment with MetaMask installed.

### 13.4.1 Connecting to a Network

```html
<!DOCTYPE html>
<html>
<head>
    <title>ethers.js Example</title>
    <script src="https://cdn.ethers.io/lib/ethers-5.7.umd.min.js"></script>
</head>
<body>
    <button id="connect">Connect Wallet</button>
    <div id="info"></div>

    <script>
        document.getElementById('connect').addEventListener('click', async () => {
            if (typeof window.ethereum !== 'undefined') {
                try {
                    // Request account access
                    await window.ethereum.request({ method: 'eth_requestAccounts' });
                    
                    // Create ethers provider
                    const provider = new ethers.providers.Web3Provider(window.ethereum);
                    const signer = provider.getSigner();
                    const address = await signer.getAddress();
                    
                    // Get network
                    const network = await provider.getNetwork();
                    
                    document.getElementById('info').innerHTML = `
                        Connected!<br>
                        Address: ${address}<br>
                        Network: ${network.name} (chainId: ${network.chainId})
                    `;
                } catch (error) {
                    console.error(error);
                }
            } else {
                alert('Please install MetaMask!');
            }
        });
    </script>
</body>
</html>
```

### 13.4.2 Reading Contract State

Let's read the total supply and balance of a known ERC-20 token (e.g., USDC on mainnet).

```javascript
const provider = new ethers.providers.JsonRpcProvider('https://eth-mainnet.g.alchemy.com/v2/YOUR_API_KEY');

const usdcAddress = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48';
const erc20ABI = [
    'function name() view returns (string)',
    'function symbol() view returns (string)',
    'function decimals() view returns (uint8)',
    'function totalSupply() view returns (uint256)',
    'function balanceOf(address) view returns (uint256)'
];

const usdcContract = new ethers.Contract(usdcAddress, erc20ABI, provider);

async function readTokenInfo() {
    const name = await usdcContract.name();
    const symbol = await usdcContract.symbol();
    const decimals = await usdcContract.decimals();
    const totalSupply = await usdcContract.totalSupply();

    console.log(`${name} (${symbol})`);
    console.log(`Decimals: ${decimals}`);
    console.log(`Total Supply: ${ethers.utils.formatUnits(totalSupply, decimals)}`);

    // Check balance of a specific address (e.g., Binance hot wallet)
    const addr = '0x28C6c06298d514Db089934071355E5743bf21d60';
    const balance = await usdcContract.balanceOf(addr);
    console.log(`Balance of ${addr}: ${ethers.utils.formatUnits(balance, decimals)}`);
}
```

### 13.4.3 Sending Transactions

Transfer some ETH to another address (requires MetaMask).

```javascript
async function sendTransaction() {
    if (typeof window.ethereum === 'undefined') return;

    const provider = new ethers.providers.Web3Provider(window.ethereum);
    const signer = provider.getSigner();

    const tx = {
        to: '0xRecipientAddressHere',
        value: ethers.utils.parseEther('0.01'), // 0.01 ETH
        gasLimit: 21000,
        maxFeePerGas: ethers.utils.parseUnits('50', 'gwei'),
        maxPriorityFeePerGas: ethers.utils.parseUnits('2', 'gwei')
    };

    try {
        const txResponse = await signer.sendTransaction(tx);
        console.log('Transaction sent:', txResponse.hash);

        // Wait for confirmation
        const receipt = await txResponse.wait();
        console.log('Transaction confirmed in block:', receipt.blockNumber);
    } catch (error) {
        console.error('Transaction failed:', error);
    }
}
```

### 13.4.4 Listening to Events

Listen for Transfer events from a contract.

```javascript
// Using WebSocket provider for real-time events
const provider = new ethers.providers.WebSocketProvider('wss://eth-mainnet.g.alchemy.com/v2/YOUR_API_KEY');

const contract = new ethers.Contract(usdcAddress, erc20ABI, provider);

// Listen for all Transfer events
contract.on('Transfer', (from, to, value, event) => {
    console.log(`
        Transfer detected:
        from: ${from}
        to: ${to}
        value: ${ethers.utils.formatUnits(value, 6)} USDC
        tx: ${event.transactionHash}
    `);
});

// Listen for specific address
contract.on('Transfer', fromAddress, toAddress, (from, to, value, event) => {
    // Filtered
});
```

### 13.4.5 Signing Messages

Sign and verify a message using MetaMask.

```javascript
async function signMessage() {
    if (!window.ethereum) return;

    const provider = new ethers.providers.Web3Provider(window.ethereum);
    const signer = provider.getSigner();
    const address = await signer.getAddress();

    const message = "Sign this message to prove you own this wallet.";

    try {
        const signature = await signer.signMessage(message);
        console.log('Signature:', signature);

        // Verify the signature
        const recoveredAddress = ethers.utils.verifyMessage(message, signature);
        console.log('Recovered address:', recoveredAddress);
        console.log('Valid signature:', recoveredAddress.toLowerCase() === address.toLowerCase());
    } catch (error) {
        console.error('Signing cancelled or failed:', error);
    }
}
```

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 13 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Web3 libraries are essential for connecting applications to    │
│  the blockchain. ethers.js and web3.js are the two main options.│
│                                                                 │
│  ethers.js FEATURES:                                           │
│  • Providers: connect to nodes (JsonRpcProvider, Web3Provider) │
│  • Signers: manage accounts (Wallet, JsonRpcSigner)            │
│  • Contracts: interact via ABI (Contract class)                │
│  • Utilities: format units, hash, sign, verify                 │
│                                                                 │
│  web3.js FEATURES:                                             │
│  • Similar concepts but with different API                     │
│  • Supports subscriptions via WebSocket                        │
│  • More mature but larger bundle                               │
│                                                                 │
│  KEY OPERATIONS:                                               │
│  • Connect wallet (eth_requestAccounts)                        │
│  • Read contract state (call)                                  │
│  • Send transactions (send)                                    │
│  • Listen to events (on)                                       │
│  • Sign and verify messages                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Key Takeaways:**
- ethers.js is recommended for most new projects due to its clean API and excellent documentation.
- Always handle user's wallet connection gracefully and handle errors.
- Use `parseEther`/`formatEther` for ETH amounts, and `parseUnits`/`formatUnits` for other tokens with custom decimals.
- For real-time features, use WebSocket providers and event listeners.
- Security: never hardcode private keys in frontend code; always use injected providers (MetaMask) for user accounts.

**Next Chapter Preview:** Chapter 14 – Building a Complete DApp. We'll combine everything learned so far—smart contracts, frontend, and Web3 libraries—to build a fully functional decentralized application, step by step.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='12. dapp_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='14. building_a_complete_dapp.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
